# Tutorial 4: Preparing QM Input & Running MiMiC

MiMiCPy is a python wrapper for preparing and executing MiMiC runs. For details on installation and compatibility, please refer to https://github.com/bharurn/mimicpy.

This is the fourth tutorial on how get MiMiCPy running, and deals with setting up the QM system and submitting MiMiC runs. Please refer to the previous tutorial on setting up and running classical MD simulation before reading this.

## QM Input Preparation

Once the system is classically minimized and equilibriated, we can start preparing the QM region. As mentioned briefly in the previous tutorial, this will require us to to use the `prepare.QM` handle. The `prepare.QM` is loaded from the saved yaml file in the working directory.

In [1]:
import mimicpy
mimicpy.setEnv(gmx='gmx_mpi')
mimicpy.setHost('server1:tests', 'source mimic.sh')
qm = mimicpy.prepare.QM.fromYaml()

Setting remote machine server1 as host..
Setting current directory to tests..
Loading session from _status.yaml..
Reading topology from /home/user/tests/prepareMM/topol.mpt..
Combining with latest coordinates data from /home/user/tests/em/em.gro..


The `prepare.QM` constructor looks for a mimicpy topology or mpt file in the folder created by `prepare.MM` (called `prepareMM` in this case). This was created when `prepare.MM.getTopology()` was called on the system. An mpt file is nothing but a Pandas dataframe stored using the Python "pickle" binary serializing protocol. It has data on the atom name, atom serial no., chain ID, etc. of each atom in the system. This format allows for the fast retrival of topology information (faster than the Gromacs TPR format). This topology is combined with the latest gro file, to get coordinate infomation.

### Adding Atoms to the QM Region

Before the QM input can be prepared, we have to tell MiMiCPy what atoms we wish to add to the QM region. This is done using the `add()` function of the `prepare.QM` handle. A special selection string is passed to efficiently select many atoms at once. This selection string has a language similar to molecular visualization softwares like VMD and PyMol.

In [2]:
qm.add('resName is SER and number < 25 and chainID not B')
qm.qmatoms

,name,resName,link,chainID,element,x,y,z
number,,,,,,,,
20,N,SER,0,A,N,11.270,7.946,6.063
21,H,SER,0,A,,11.188,7.894,6.090
22,CA,SER,0,A,C,11.387,7.864,6.024
23,HA,SER,0,A,,11.471,7.930,6.004
24,CB,SER,0,A,C,11.433,7.771,6.137


The following is a brief summary of the selection langauge:

- Each selection is a set of three words
- The first word is selection name. It can be any of the following:
    - number, name, resName, chainID, element, x, y, z
- The second word is an operator. It can be any of the following:
    - is (for equality), <, >, >=, <=, not (for negeation)
- The third is the actual value to match the selection with. It can be an integer, float or string depending on the selection name.
- Any number of selection sets can be strung together with the 'and' or 'or' logical operators.

The `add()` can be called any number of times before the `getInp()` is called to generate the input script.

#### Link Atoms

If the QM-MM interface cuts across covalent bonds, then link atoms would need to be specified. Please refer to the CPMD manual for more details on this. To inform MiMiCPy that the atoms being added are in link atoms, all you need to do is pass the `link=True` flag when adding atoms; for example, `qm.add('name CA', link=True)` will add all C-alpha carbons as link atoms.

### Generate the QM Input Script

Now we are ready to prepare the QM input script. For this we use the `getInp()` function of the `prepaare.QM` handle. It takes a Gromacs MDP script as input. This will be used to generated the Gromacs TPR file that will serve as the run file for the MiMiC run. `getInp()` sets the `MAXSTEPS` and `TIMESTEP` parameters in CPMD QM input script from the `nsteps` and `dt` parameters of the mdp script.

In [3]:
test = mimicpy.utils.scripts.mdp.MDP.defaultEM()
test.dt = 0.0001
test.nsteps = 50000
qm.getInp(test)

Changing Gromacs integrator to MiMiC..
Writing atoms in QM region to index.ndx..
Generating Gromacs tpr file for MiMiC run..
Running gmx_mpi -quiet grompp -f mimic.mdp -c /home/users/tests/em/em.gro -p /home/users/tests/prepareMM/topol.top -o mimic.tpr -pp processed.top -n index.ndx..
Reading force field data to fill in charges and missing atomic symbol information..
Creating CPMD input script..
Saving status to _status.yaml..


The `getInp()` function performs the following actions:
- Writes all atoms added to the QM region to an index file, and feeds it to Gromacs
- Generates the TPR file for the MiMiC run, using coordinate data from the latest coordinate file
- A preprocessed topology is also generated, which contains all the information about the force field parameters. This is used to calculate the total charge of the QM region, and also fill in the atomic numbers for which information was not available in the input PDB.
- The `ATOM` section of the CPMD QM input script is generated, as well as the `MIMIC`, `CPMD`, and `SYSTEM` sections.

The generated script is returned by the function. It is also accesible by the `inp` property of the `prepare.QM` object. As with other scripts, the contents can be seen by converting the object into a string.

In [4]:
print(qm.inp)


&INFO
MiMiC Run
&END

&MIMIC
PATHS
1
---
BOX
270.2385837210658  270.2385837210658  270.2385837210658
OVERLAPS
5
2 22 1 1
2 24 1 2
2 21 1 3
2 23 1 4
2 20 1 5
&END

&SYSTEM
CELL
17  0.88  0.83
CHARGE
0.7
&END

&CPMD
MIMIC
PARALLEL CONSTRAINTS
MAXSTEPS
50000
TIMESTEP
4
&END

&ATOMS
*C_MT_BLYP
   LMAX=P
    2
  215.18311382902846   148.60806245292702   113.83710175692171
  216.0523878464286   146.8506171568789   115.97249227792636

*H_MT_BLYP
   LMAX=P
    2
  211.4225588407105   149.17498029036187   115.08432099927842
  216.7704837738461   149.8552816952837   113.45915653196512

*N_MT_BLYP
   LMAX=P
    1
  212.97213426303247   150.15763787524898   114.57409494558703

&END



The most important section of the script are generated by the `getInp()` function. However, other options specific to the your particular MiMiC run have to be set by you. This can be easily done. For example, to set the temperate run `inp.cpmd.temperature = 300` or to set the poission solver tuckerman option run `inp.system.poission__solver__tuckerman = ''`. For more information on how to change the CPMD script parameters, refer to the tutorial on scripts.

## Running MiMiC

Now we are ready to run a MiMiC simulation. We can run it on the current node. However, it is recommended to use Slurm to run MiMiC jobs. So, first we create a jobscript. In this case, we load a previous jobscript.

In [5]:
jbs = mimicpy.scripts.Slurm.loadFromFile('jobscript.sh')

And change a few parameters.

In [6]:
jbs.clearCommands()
jbs.time = '01:00:00'
jbs.account = "TEST"
print(jbs)

#!/bin/bash -x
#SBATCH --nodes=3
#SBATCH --cpus-per-task=4
#SBATCH --error=mpi_err.%j
#SBATCH --time=01:00:00
#SBATCH --partition=batch
#SBATCH --account=TEST
#SBATCH --job-name=jobscript
#SBATCH --output=jobscript.%J.out




Then we create a `simulate.MiMiC` handle, continuing from the previous QM handle. We also set its jobscript settings using the jobscript we loaded above.

In [7]:
mimic = mimicpy.simulate.MiMiC.continueFrom(qm)
mimic.setSlurmSettings(jbs)

Setting Slurm job settings from jobscript jobscript..
Transferring sources, modules and other header commands from host to job script..


MiMiC requires both Gromacs and CPMD to run parallel. For efficient execution, we need to specificy the settings for each run explicitly. This can be done as shown below

In [8]:
mimic.setGMXSettings(N=1, ntasks_per_node=6, cpus_per_task=4, r=0)
mimic.setCPMDSettings(N=2, ntasks_per_node=6, cpus_per_task=4, r=0)

The CPMD pseudopotential path is set, and then we call the `run()` function to start the MiMiC run.

In [9]:
mimicpy.setExec(cpmd_pp='/bin/CPMD/PP')
mimic.run(qm.inp)
mimicpy.closeHost()

MiMiC run submmitted as a Slurm job jobscript.sh with the job ID 1382412..
The host and/or this script can be safely closed..